# 3. Logistic Regression
### 3.1. Model Definition (two class)
Transforms continuous value from linear regression into discrete values for output.

$$P(Y=1|X=\{X_1,\ldots, X_n\})=\frac{1}{1+\exp{(w_0 + \sum_{i=1}^n w_iX_i)}}$$

$$P(Y=0|X=\{X_1,\ldots, X_n\})=\frac{\exp{(w_0 + \sum_{i=1}^n w_iX_i)}}{1+\exp{(w_0 + \sum_{i=1}^n w_iX_i)}}$$

### 3.2. MCLE (two class)
We want to use MCLE to learn the model parameters. MCLE cannot be solved in closed-form.

$$\hat{W}_{MCLE}=\underset{W}{argmax} \prod_l P(Y^l|X^l,W)$$

where
- $l$: number of training examples

### 3.3. MCLE (two class) with Gradient Ascent
Parameters can be derived using gradient descent because logistic regression is concave. Start with a random initialization of parameters. Repeat until the change is less than $\epsilon$, that is, until $l(W)_t - l(W)_{t-1} < \epsilon$.

$$l(W)=\sum_l Y^l(w_0+\sum_i^n w_i X^l_i)-ln(1+exp(w_0+\sum_i^n w_i X^l_i))$$

$$w_i \leftarrow w_i + \eta \sum_l X^l_i\left(Y^l-\hat{P}(Y^l=1|X^l,W)\right) $$

where 
- $\eta$ is step size (learning rate)
- $ X^l_i\left(Y^l-\hat{P}(Y^l=1|X^l,W)\right)$ is the gradient
- $\hat{P}(Y^l=1|X^l,W)=\frac{exp(w_0+\sum_iw_iX_i)}{1+exp(w_0+\sum_iw_iX_i)}$

***


### 3.4. MAP estimation (two class) with Gradient Ascent
Regularization term helps reduce overfitting, especially when training data is sparse.

$$l(W)=?$$

$$w_i \leftarrow w_i -\eta\lambda w_i + \eta \sum_l X^l_i\left(Y^l-\hat{P}(Y^l=1|X^l,W)\right) $$

where
- $\lambda$ is a regularization term, $\lambda=\frac{1}{2\sigma^2}$

### 3.4. Model Definition (multiclass)
Learn R-1 set of weights.

For k<R:

$$P(Y=y_k|X=\{X_1,\ldots, X_n\})=\frac{\exp{(w_{k,0} + \sum_{i=1}^n w_{k,i}X_i)}}{1+\sum_{j=1}^{R-1}\exp{(w_{j,0} + \sum_{i=1}^n w_{ji}X_i)}}$$

where
- R: number of classes

***

For k=R:

$$P(Y=y_R|X=\{X_1,\ldots, X_n\})=\frac{1}{1+\sum_{j=1}^{R-1}\exp{(w_{j,0} + \sum_{i=1}^n w_{ji}X_i)}}$$